![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/ocr/PDF_TEXT_NER.ipynb)

# Recognize entities in scanned PDFs

To run this yourself, you will need to upload your **Spark OCR** license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

For more in-depth tutorials: https://github.com/JohnSnowLabs/spark-ocr-workshop/tree/master/jupyter

## 1. Colab Setup

Install correct version of Pillow and Restart runtime

In [1]:
# Install correct Pillow version
import PIL
if PIL.__version__  != '6.2.1':
  print ('Installing correct version of Pillow. Kernel will restart automatically')
  !pip install --upgrade pillow==6.2.1
  # hard restart runtime
  import os
  os.kill(os.getpid(), 9)
else:
  print ('Correct Pillow detected')

Correct Pillow detected


Read licence key

In [5]:
import os
import json

with open('workshop_license_keys.json') as f:
    license_keys = json.load(f)

print (license_keys.keys())

secret = license_keys['JSL_OCR_SECRET']
os.environ['SPARK_OCR_LICENSE'] = license_keys['SPARK_OCR_LICENSE']
os.environ['JSL_OCR_LICENSE'] = license_keys['SPARK_OCR_LICENSE']
version = secret.split("-")[0]
nlp_secret = license_keys['SECRET']
jsl_version = nlp_secret.split('-')[0]
print ('Spark OCR Version:', version)

dict_keys(['SECRET', 'JSL_OCR_SECRET', 'SPARK_OCR_LICENSE'])
Spark OCR Version: 1.5.0


Install Dependencies

In [ ]:
# Install Java
!apt-get update
!apt-get install -y openjdk-8-jdk
!java -version

# Install pyspark, SparkOCR, and SparkNLP
!pip install --ignore-installed -q pyspark==2.4.4
# Insall Spark Ocr from pypi using secret
!python -m pip install --upgrade spark-ocr==$version  --extra-index-url https://pypi.johnsnowlabs.com/$secret
# or install from local path
# %pip install --user ../../python/dist/spark-ocr-[version].tar.gz

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$nlp_secret


Importing Libraries

In [6]:
import pandas as pd
import numpy as np
import os

#Pyspark Imports
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Necessary imports from Spark OCR library
from sparkocr import start
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import display_image, to_pil_image
from sparkocr.metrics import score
import pkg_resources


# import sparknlp packages
import sparknlp_jsl
from sparknlp_jsl.annotator import *

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]


Start Spark Session

In [7]:
spark = start(secret=secret,
              nlp_secret=nlp_secret,
              nlp_version=jsl_version,
              nlp_internal=True)

spark

## 2. Download and read scanned pdf image

In [13]:
!wget https://www.reneelab.com/wp-content/uploads/sites/2/2015/11/target-500x600.png -O 1.jpg

--2020-08-10 17:50:38--  https://www.reneelab.com/wp-content/uploads/sites/2/2015/11/target-500x600.png
Resolving www.reneelab.com (www.reneelab.com)... 172.67.74.197, 104.26.12.76, 104.26.13.76, ...
Connecting to www.reneelab.com (www.reneelab.com)|172.67.74.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/png]
Saving to: ‘1.jpg’

1.jpg                   [ <=>                ] 141.88K  --.-KB/s    in 0.009s  

2020-08-10 17:50:38 (14.8 MB/s) - ‘1.jpg’ saved [145284]



In [11]:
image_df = spark.read.format("binaryFile").load('./1.jpg').cache()
image_df.show()

+-------------------+-------------------+------+--------------------+
|               path|   modificationTime|length|             content|
+-------------------+-------------------+------+--------------------+
|file:/content/1.jpg|2016-12-19 13:28:45|145284|[89 50 4E 47 0D 0...|
+-------------------+-------------------+------+--------------------+



## 3. Construct the OCR pipeline

In [26]:
# Read binary as image
binary_to_image = BinaryToImage()
binary_to_image.setInputCol('content')
binary_to_image.setOutputCol('image')

# Scale image
scaler = ImageScaler()
scaler.setInputCol('image')
scaler.setOutputCol('scaled_image')
scaler.setScaleFactor(2.0)

# Binarize using adaptive tresholding
binarizer = ImageAdaptiveThresholding()
binarizer.setInputCol('scaled_image')
binarizer.setOutputCol('binarized_image')
binarizer.setBlockSize(91)
binarizer.setOffset(70)

# Remove extraneous objects from image
remove_objects = ImageRemoveObjects()
remove_objects.setInputCol('binarized_image')
remove_objects.setOutputCol('cleared_image')
remove_objects.setMinSizeObject(30)
remove_objects.setMaxSizeObject(4000)

# Apply morphology opening
morpholy_operation = ImageMorphologyOperation()
morpholy_operation.setKernelShape(KernelShape.DISK)
morpholy_operation.setKernelSize(1)
morpholy_operation.setOperation('closing')
morpholy_operation.setInputCol('cleared_image')
morpholy_operation.setOutputCol('corrected_image')

# Extract text from corrected image with OCR
ocr = ImageToText()
ocr.setInputCol('binarized_image')
ocr.setOutputCol('text')
ocr.setConfidenceThreshold(50)
ocr.setIgnoreResolution(False)

documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
    
embeddings = WordEmbeddingsModel.pretrained('glove_100d').\
                    setInputCols(["document", 'token']).\
                    setOutputCol("embeddings")

public_ner = NerDLModel.pretrained('onto_100', 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_converter = NerConverter() \
                .setInputCols(["document", "token", "ner"]) \
                  .setOutputCol("ner_chunk")

# Create pipeline
pipeline = PipelineModel(stages=[
    binary_to_image,
    scaler,
    binarizer,
    remove_objects,
    morpholy_operation,
    ocr])

nlp_pipeline =  Pipeline(stages=[documentAssembler, 
    tokenizer,
    embeddings,
    public_ner,
    ner_converter])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


## 4. Run OCR pipeline

In [27]:
result = pipeline.transform(image_df).cache()

## 5. Visualize Results

Display result dataframe

In [28]:
result.select("text", "confidence").show()

+--------------------+----------------+
|                text|      confidence|
+--------------------+----------------+
|ADVERTISEMENT.

T...|91.8821029663086|
+--------------------+----------------+



Display text and images

In [29]:
result_arr = []
for r in result.distinct().collect():
  print (r.text)
  result_arr.append(r.text)

ADVERTISEMENT.

Tuts publication of the Works of Jonn Kwox, it is
supposed, will extend to Five Volumes. It was thought
advisable to commence the series with his History of
the Reformation in Scotland, as the work of greatest
importance. The next volume will thus contain the
Third and Fourth Books, which continue the History to
the year 1564; at which period his historical labours
may be considered to terminate. But the Fifth Book,
forming a sequel to the History, and published under
his name in 1644, will also be included. His Letters
and Miscellancous Writings will be arranged in the
subsequent volumes, as nearly as possible in chronolo-
gical order; each portion being introduced by a separate
avtice, respecting the manuscript or printed copies from
which they have been taken.

It may perhaps be expected that a Life of the Author
thould have been prefixed to this volume. The Life of
Knox, by Dr. M-Crig, is however a work so universally
known, and of so much historical value, as to su

# 6. Run NER pipeline

In [30]:
empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":result_arr}))
nlp_result = pipelineModel.transform(df)

#7. Visualize NER results

In [31]:

nlp_result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)


+--------------------------------------+-----------+
|chunk                                 |ner_label  |
+--------------------------------------+-----------+
|Tuts                                  |PERSON     |
|the Works of Jonn Kwox                |WORK_OF_ART|
|Five Volumes                          |WORK_OF_ART|
|History of
the Reformation            |EVENT      |
|Scotland                              |GPE        |
|Third                                 |ORDINAL    |
|Fourth                                |ORDINAL    |
|History to
the year 1564              |DATE       |
|Fifth                                 |ORDINAL    |
|History                               |WORK_OF_ART|
|1644                                  |DATE       |
|His Letters
and Miscellancous Writings|WORK_OF_ART|
|The Life of
Knox                      |WORK_OF_ART|
+--------------------------------------+-----------+

